<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/langsmith/tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangSmith Tracing 快速入门

为了使用追踪（Tracing）功能，必须将`LANGCHAIN_TRACING_V2`环境变量设置为`true`，以便在使用 @traceable或traceable时将跟踪日志记录到LangSmith。使得开发者可以在不更改代码的情况下切换跟踪开关。

此外，还需要将 `LANGCHAIN_API_KEY` 环境变量设置好（注册和创建`API_KEY`官方文档链接：https://docs.smith.langchain.com/）

### 追踪记录方法

本教程将展示多种使用 LangSmith 来记录追踪 LLM 生成内容和性能的方法：
- 使用 `@traceable` 装饰器追踪特定 Python 函数
- 使用 `wrap_openai` 方法自动追踪 OpenAI 客户端所有调用
- 使用 `RunTree` API
- 使用 `trace` 上下文管理
- 结合 `LangChain` 来追踪记录

### 记录到特定项目

追踪日志默认会记录在 `default` 项目中，如果想要记录在其他项目，需要在 LangSmith 平台新建项目。然后使用以下方式：
- 使用环境变量全量修改：`export LANGCHAIN_PROJECT=my-custom-project`
- 动态修改单条记录：`@traceable(project_name="my-custom-project")`


## @traceable 装饰器

In [12]:
from langsmith import traceable
from openai import Client

# 创建 OpenAI 客户端
# openai = Client()
# # OpenAI API调用（代理方式）
# openai = Client(
#     api_key="XXX",
#     base_url="https://vip.apiyi.com/v1"
# )

# # 智谱API调用
# openai = Client(
#     api_key="XXX",
#     base_url="https://open.bigmodel.cn/api/paas/v4/"
# )

# DeepSeek API调用
openai = Client(
    api_key="XXX",
    base_url="https://api.deepseek.com"
)

# 标记函数可追踪
@traceable
def format_prompt(subject):
    # 格式化提示信息
    return [
        {
            "role": "system",
            "content": "你是一个乐于助人的助手。",
        },
        {
            "role": "user",
            "content": f"对于一家卖{subject}的店来说，取个什么名字好呢？"
        }
    ]

# 标记函数可追踪，并指定运行类型为 LLM
@traceable(run_type="llm")
def invoke_llm(messages):
    # 调用 OpenAI 的聊天模型
    return openai.chat.completions.create(
        # OpenAI API
        # messages=messages, model="gpt-3.5-turbo", temperature=0
        # 智谱API
        #messages=messages, model="glm-4v-flash", temperature=0
        # DeepSeek API调用
        messages=messages, model="deepseek-chat", temperature=0
    )

# 标记函数可追踪
@traceable
def parse_output(response):
    # 解析并返回模型的输出
    return response.choices[0].message.content

# 标记函数可追踪
@traceable
def run_pipeline(prompt):
    # 运行整个管道流程
    messages = format_prompt(prompt)  # 创建提示信息
    response = invoke_llm(messages)  # 调用模型
    return parse_output(response)  # 解析模型输出

In [13]:
run_pipeline("烤鸭")

'给一家烤鸭店取名字时，既要突出烤鸭的特色，又要让人印象深刻，同时还要有文化内涵或趣味性。以下是一些建议：\n\n### 1. **传统风格**\n   - **全聚德**（已有知名品牌，但可以参考这种风格）\n   - **京味轩**：突出北京烤鸭的地域特色。\n   - **御鸭坊**：带有皇家气息，显得高档。\n   - **老北京烤鸭**：简单直接，突出传统风味。\n   - **鸭香阁**：古典雅致，适合传统风格的店铺。\n\n### 2. **现代风格**\n   - **鸭先生**：时尚、亲切，适合年轻消费者。\n   - **鸭掌门**：霸气且有特色，容易记住。\n   - **鸭香四溢**：突出烤鸭的香气，吸引顾客。\n   - **鸭趣**：轻松有趣，适合年轻化的品牌。\n   - **鸭小厨**：亲切、接地气，适合小型或连锁店。\n\n### 3. **创意风格**\n   - **鸭来了**：简单有趣，容易让人记住。\n   - **鸭香不怕巷子深**：借用俗语，突出烤鸭的美味。\n   - **鸭香十里**：夸张地表达烤鸭的香气，吸引顾客。\n   - **鸭香满堂**：寓意生意兴隆，顾客满堂。\n   - **鸭香传奇**：带有故事感，吸引顾客的好奇心。\n\n### 4. **地域特色**\n   - **京鸭坊**：突出北京烤鸭的地域特色。\n   - **金陵鸭香**：如果店铺在南京，可以突出金陵文化。\n   - **蜀香鸭**：如果结合川味，可以突出麻辣风味。\n   - **江南鸭韵**：适合南方风格的烤鸭店。\n\n### 5. **高端风格**\n   - **御品鸭香**：高端大气，适合高档餐厅。\n   - **金鸭阁**：带有奢华感，适合高端消费群体。\n   - **皇家鸭宴**：突出皇家风范，适合高档餐厅。\n   - **鸭香御府**：带有贵族气息，适合高端品牌。\n\n### 6. **趣味风格**\n   - **鸭力山大**：谐音“压力山大”，轻松幽默。\n   - **鸭香不怕胖**：幽默地表达烤鸭的美味，吸引年轻人。\n   - **鸭香不怕晚**：适合夜宵或晚市，突出随时可以享受美味。\n   - **鸭香不怕等**：突出烤鸭的美味值得等待。\n\n### 7. **结合店铺特色**\n   - 如

In [ ]:
run_pipeline("驴肉火烧")

'给一家卖驴肉火烧的店取名时，既要突出特色，又要朗朗上口，容易让顾客记住。以下是一些建议：\n\n1. **驴香记**  \n   突出驴肉的香味，简洁大气，容易让人联想到美味的驴肉火烧。\n\n2. **火烧驴坊**  \n   直接点明店铺的主打产品，简单明了，容易吸引顾客。\n\n3. **驴火传奇**  \n   给人一种历史悠久、品质上乘的感觉，适合想要打造品牌形象的店铺。\n\n4. **驴香四溢**  \n   强调驴肉的香气，给人一种食欲感，适合吸引食客。\n\n5. **驴肉火烧王**  \n   简单直接，突出店铺的专业性和权威性，适合想要打造“王者”形象的店铺。\n\n6. **驴香满堂**  \n   给人一种热闹、丰盛的感觉，适合营造温馨的用餐氛围。\n\n7. **驴火飘香**  \n   强调火烧的香气，吸引顾客进店品尝。\n\n8. **驴肉火烧铺**  \n   简单直接，适合走亲民路线的店铺。\n\n9. **驴香坊**  \n   简洁大方，突出驴肉的特色，适合小而精致的店铺。\n\n10. **驴火人家**  \n    给人一种亲切感，适合想要营造家庭氛围的店铺。\n\n11. **驴香一品**  \n    强调驴肉火烧的高品质，适合走高端路线的店铺。\n\n12. **驴火飘香坊**  \n    结合了“驴火”和“飘香”，既突出产品，又让人联想到美味。\n\n13. **驴肉火烧世家**  \n    给人一种传承已久的感觉，适合有历史背景的店铺。\n\n14. **驴香阁**  \n    带有古典气息，适合想要打造文化氛围的店铺。\n\n15. **驴火香缘**  \n    强调与顾客的缘分，适合想要营造温馨氛围的店铺。\n\n在选择名字时，建议结合店铺的定位、目标顾客群体以及店铺的风格，确保名字既能吸引顾客，又能传达出店铺的特色和理念。'

## wrap_openai 客户端

Python/TypeScript 中的 wrap_openai/wrapOpenAI 方法允许您包装 OpenAI 客户端，以便自动记录跟踪 - 无需装饰器或函数包装！

该包装器与 @traceable 装饰器或可追溯函数完美配合，可以在同一应用程序中同时使用两者。

In [15]:
import openai
from langsmith import traceable
from langsmith.wrappers import wrap_openai

# 包装 OpenAI 客户端
client = wrap_openai(openai.Client())

# 标记函数可追踪，并指定运行类型为工具，名称为"Retrieve Context"
@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    # 返回上下文信息
    return "在今天早上的会议中，我们讨论了出海创业的机会与挑战。"

# 标记函数可追踪，名称为"Chat Pipeline"
@traceable(name="Chat Pipeline")
def chat_pipeline(question: str):
    # 获取上下文信息
    context = my_tool(question)

    # 创建消息列表，包含系统消息和用户消息
    messages = [
        { "role": "system", "content": "你是一个乐于助人的助手。请仅根据给定的上下文回复用户的请求。" },
        { "role": "user", "content": f"问题：{question}\n上下文：{context}"}
    ]

    # 调用聊天模型生成回复
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages
    )

    # 返回模型的回复内容
    return chat_completion.choices[0].message.content

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# 调用 chat_pipeline 函数
chat_pipeline("你能总结一下今天早上的会议吗？")

'当然！在今天早上的会议中，我们主要讨论了出海创业的机会与挑战。'

## RunTree API

通过 RunTree API，是跟踪日志记录到 LangSmith 的另一种更直接的方式。

该API允许对跟踪进行更多控制：可以手动创建运行和子运行以组装您的跟踪。

您仍然需要设置LANGCHAIN_API_KEY，但对于**此方法不需要LANGCHAIN_TRACING_V2**。


In [ ]:
import openai
from langsmith.run_trees import RunTree

# 用户输入的问题
question = "你能总结一下今天早上的会议吗？"

# 创建一个顶层的运行节点
pipeline = RunTree(
    name="Chat Pipeline",
    run_type="chain",
    inputs={"question": question}
)

# 在检索步骤中获取的上下文信息
context = "在今天早上的会议中，我们回顾了改革开放的重大成果。"

# 创建消息列表，包含系统消息和用户消息
messages = [
    { "role": "system", "content": "你是一个乐于助人的助手。请仅根据给定的上下文回复用户的请求。" },
    { "role": "user", "content": f"问题：{question}\n上下文：{context}"}
]

# 创建一个子运行节点
child_llm_run = pipeline.create_child(
    name="OpenAI Call",
    run_type="llm",
    inputs={"messages": messages},
)

# 生成回复
client = openai.Client()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
)

# 结束子运行节点并记录输出
child_llm_run.end(outputs=chat_completion)
child_llm_run.post()

# 结束顶层运行节点并记录输出
pipeline.end(outputs={"answer": chat_completion.choices[0].message.content})
pipeline.post()

## trace 上下文管理器

在Python中，使用trace上下文管理器跟踪日志记录到LangSmith。在以下情况下非常有用：

- 想要为特定代码块记录跟踪日志，而不设置一个会为整个应用程序记录跟踪的环境变量。
- 希望对跟踪的输入、输出和其他属性进行控制。
- 使用装饰器或包装器并不可行。

该上下文管理器与可追溯的装饰器和 `wrap_openai` 包装器无缝集成，因此，可以在同一应用程序中同时使用它们。

In [ ]:
import openai
from langsmith import trace
from langsmith import traceable
from langsmith.wrappers import wrap_openai

# 包装 OpenAI 客户端
client = wrap_openai(openai.Client())

# 标记函数可追踪，并指定运行类型为工具，名称为"Retrieve Context"
@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    # 返回上下文信息
    return "During this morning's meeting, we solved all world conflict."

def chat_pipeline(question: str):
    # 获取上下文信息
    context = my_tool(question)

    # 创建消息列表，包含系统消息和用户消息
    messages = [
        { "role": "system", "content": "你是一个乐于助人的助手。请仅根据给定的上下文回复用户的请求。" },
        { "role": "user", "content": f"问题：{question}\n上下文：{context}"}
    ]

    # 调用聊天模型生成回复
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages
    )

    # 返回模型的回复内容
    return chat_completion.choices[0].message.content

# 应用输入
app_inputs = {"input": "Can you summarize this morning's meetings?"}

# 跟踪聊天管道运行
with trace("Chat Pipeline", "chain", project_name="my_test", inputs=app_inputs) as rt:
    output = chat_pipeline("Can you summarize this morning's meetings?")
    rt.end(outputs={"output": output})


## 记录多模态模型 GPT-4

In [ ]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

# 包装 OpenAI 客户端
client = wrap_openai(OpenAI())

# 调用聊天模型生成回复
response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图讲的什么？"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg",
          },
        },
      ],
    }
  ],
)

In [ ]:
# 打印回复的内容
print(response.choices[0].message.content)

这幅图是一种幽默搞笑的对比图。左侧展示的是一只形如肌肉男的柴犬，被称为“16岁的我”，右侧则是一只普通的柴犬，被称为“工作后的我”。图片通过夸张的肌肉和普通的狗的形态来幽默地表达了人们对比自己年轻时充满活力和成年后工作压力导致身体和精神状态“变形”的感受。左边的大肌肉柴犬下方的文字翻译为“我可以一口气做一百个俯卧撑，一条跑足十公里，浴火重生的女人，人见人爱的大男孩”，而右边的普通柴犬下方的文字翻译为“好累啊 好想赖床 浑身疼痛 我没有病 你心有病 我命由我不由天 独步天下”。这些标签富含讽刺和幽默意味，反映了现代生活中劳累与压力的普遍现象。


## 结合 LangChain 记录追踪

设置好以下环境变量后，无需任何额外的代码即可追踪 LangChain 运行
```shell
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=<your-api-key>

# The below examples use the OpenAI API, so you will need
export OPENAI_API_KEY=<your-openai-api-key>
```

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 从消息中创建聊天提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手。请参考给定的上下文回复用户的请求。"),
    ("user", "问题：{question}\n上下文：{context}")
])

# 使用指定的模型
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

# 将提示模板、模型和输出解析器链在一起
chain = prompt | model | output_parser

# 定义问题和上下文
question = "孙悟空到底打过几次白骨精啊？"
context = "其实孙悟空三打白骨精后，又打了她一次"

In [ ]:
# 调用链条并传递输入
chain.invoke({"question": question, "context": context})

'孙悟空一共打过四次白骨精。'